# Βοηθητικό υλικό για την εργασία στο μάθημα AIDA 

******
Δεν είναι απαραίτητο να χρησιμοποιήσετε τον κώδικα ή τις τεχνικές που παρουσιάζονται στο παρών notebook. Το υλικό αυτό δίνεται ως βοήθημα και μπορεί να χρησιμοποιηθεί ως έχει ή να το προσαρμόσετε όπως νομίζετε ή να μην το χρησιμοποιήσετε καθόλου στην δική σας υλοποίηση.

Ο κώδικας αποτελεί μέρος του project που βρίσκεται σε [αυτό](https://github.com/geofila/Conceptual-Edits-as-Counterfactual-Explanations) το αποθετήριο.
******
## Κατασκευή του Dataset


1. Αρχικά πρέπει να κατεβάσουμε το dataset από το site και στην συνέχεια να το αποσυμπιέσουμε ώστε να χρησιμοποιήσουμε το json αρχείο που περιέχει πληροφορίες σχετικά με τις εικόνες.
2. Στην συνέχεια θα διαβάσουμε τις πληροφορίες που περιέχονται στο το json.
3. Θα φιλτράρουμε τις εικόνες με βάση τα αντικείμενα που θέλουμε να περιέχουν για να ολοκληρώσουμε την κατασκευή του Dataset. Για αυτό παρέχετε ένα αντικείμενο με το όνομα COCO το όποιο δέχεται στην είσοδο τα δεδομένα από το json αρχείο. Η κλάση αυτή διατηρεί τις πληροφορίες της σε ένα dictonary με το όνομα coco, το όποιο περιέχει πληροφορίες αρχικά για όλες τις εικόνες. Επίσης έχει μια μέθοδο prune_dataset η όποια δέχεται στην είσοδο μια λίστα από strings με τα ονόματα των αντικειμένων τα όποια θέλουμε να περιέχονται στις εικόνες. Η μέθοδος αυτή κλαδεύει τις εικόνες από το dictionary του cocο. Οι τελικές εικόνες που θα έχει το dictonary coco θα είναι οι εικόνες που περιέχουν όλα τα αντικείμενα που δόθηκαν στην λίστα (ή και περισσότερα). Οπότε αν η μέθοδος κληθεί με ορίσματα `[cat, dog]` τότε μείνουν μόνο οι εικόνες που περιέχουν σίγουρα και cat και dog και μπορεί και παραπάνω αντικείμενα π.χ. book.

In [ ]:
# download dataset
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip # download dataset
!unzip annotations_trainval2014.zip # unzip dataset

--2022-07-10 16:17:57--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.33.169
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.33.169|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  70.8MB/s    in 3.4s    

2022-07-10 16:18:00 (70.8 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]

Archive:  annotations_trainval2014.zip
  inflating: annotations/instances_train2014.json  

In [ ]:
import json 
from tqdm.notebook import tqdm
# Load Json File with annotations
with open("/content/annotations/instances_train2014.json", "rb") as f:
    data = json.load(f)

JSONDecodeError: ignored

In [ ]:
def is_sublist(list1, list2):
    l1 = list1.copy()
    l2 = list2.copy()
    for l in l1:
        if l in l2:
            l2.remove(l)
        else:
            return False
    return True

class COCO:
    
    def __init__(self, json_data):
        self.json_data = json_data
        self.categories = {}
        self.coco = {}
        for im in self.json_data["images"]:
            self.coco[im["id"]] = {"url": im["flickr_url"]} # αρχικά αποθηκευούμε για κάθε εικόνα το url της ώστε να μπορέσουμε να το εκτυπώσουμε
    
        for cat in self.json_data["categories"]:
            self.categories[cat["id"]] = {"name": cat["name"], "supercategory": cat["supercategory"]} 

        anns_per_image = {}
        print ("Loading Coco Metadata")
        for ann in tqdm(self.json_data["annotations"]):
            if ann["image_id"] not in anns_per_image:
                anns_per_image[ann["image_id"]] = {"label_ids": [ann["category_id"]], "label_texts": [self.categories[ann["category_id"]]["name"]]}
            else:
                anns_per_image[ann["image_id"]]["label_ids"].append(ann["category_id"])
                anns_per_image[ann["image_id"]]["label_texts"].append(self.categories[ann["category_id"]]["name"])


        # ενώνουμε τις κατηγορίες με το dict των εικόνων 
        for im_id in list(self.coco):
            if im_id in anns_per_image:
                self.coco[im_id]["label_ids"] = anns_per_image[im_id]["label_ids"]
                self.coco[im_id]["label_texts"] = anns_per_image[im_id]["label_texts"]
            else:
                self.coco.pop(im_id, None) # αν δεν έχουμε annotations σβήνουμε το key αυτό 



    def prune_dataset(self, labels):
        image_ids = list(self.coco.keys())
        for image_id in image_ids:
            if not is_sublist(labels, self.coco[image_id]["label_texts"]):
                self.coco.pop(image_id, None)
coco = COCO(data)


print (f"COCO keys: {coco.coco[list(coco.coco.keys())[0]].keys()}")

In [ ]:
print (f"Size of dataset before Pruning: {len(coco.coco)}")
coco.prune_dataset(["chair"])
print (f"Size of dataset after Pruning: {len(coco.coco)}")

In [ ]:
len(coco.coco)


## Places Predictor 


Παρακάτω σας δίνεται μια κλάση για η όποια υλοποιεί τον classifier που δόθηκε μαζί με την [δημοσιευση](http://places2.csail.mit.edu/) του Places Dataset από το MIT. Στην παρούσα εργασία καλείστε να τρέξετε τον ταξινομητή αυτό για τις εικόνες που επιλέξατε από το coco και με βάση τα prediction τους να προσπαθήσετε να εξηγήσετε την λειτουργία του ταξινομητή.  Η κλάση αυτή περιέχει μια μέθοδο `classify` η όποια δέχεται στην είσοδο ένα url και επιστρέφει τα 5 πρώτα prediction του classifier μαζί με τις αντίστοιχες πιθανότητες. 

Τέλος σας δίνεται και μια συνάρτηση ```read_image_from_url```  για το διάβασμα εικόνων από urls, ώστε να μπορείτε να τυπώνετε εικόνες στο περιβάλλον του notebook. 

In [ ]:
from skimage import io

def read_image_from_url(img_url):
    img = io.imread(img_url)
    img = Image.fromarray(img)
    return img

In [ ]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
from matplotlib import pyplot as plt

class Classifier:

    def __init__(self, arch):
        # th architecture to use
        self.arch = arch

        # load the pre-trained weights
        self.model_file = '%s_places365.pth.tar' % self.arch
        if not os.access(self.model_file, os.W_OK):
            weight_url = 'http://places2.csail.mit.edu/models_places365/' + self.model_file
            os.system('wget ' + weight_url)

        self.model = models.__dict__[arch](num_classes=365)
        checkpoint = torch.load(self.model_file, map_location=lambda storage, loc: storage)
        state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
        self.model.load_state_dict(state_dict)
        self.model.eval()


        # load the image transformer
        self.centre_crop = trn.Compose([
                trn.Resize((256,256)),
                trn.CenterCrop(224),
                trn.ToTensor(),
                trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # load the class label
        file_name = 'categories_places365.txt'
        if not os.access(file_name, os.W_OK):
            synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
            os.system('wget ' + synset_url)
        
        self.classes = list()
        with open(file_name) as class_file:
            for line in class_file:
                self.classes.append(line.strip().split(' ')[0][3:])
        self.classes = tuple(self.classes)


    # load the test image
    def classify(self, img_url):
      img = io.imread(img_url)
      img = Image.fromarray(img)
      input_img = V(self.centre_crop(img).unsqueeze(0))

      # forward pass
      logit = self.model.forward(input_img)
      h_x = F.softmax(logit, 1).data.squeeze()
      probs, idx = h_x.sort(0, True)
      # output the prediction
      preds = []
      for i in range(0, 5):
          preds.append([self.classes[idx[i]], float(probs[i])])
      return preds

In [ ]:
predictor = Classifier("resnet18")

In [ ]:
image_url = "https://farm5.staticflickr.com/4141/4909108695_62224087a9_z.jpg"
print (f"Image url: {image_url}")
image = read_image_from_url(image_url)
plt.imshow(image)

print ("Predictions")
predictor.classify(image_url)

# Γνώση και wordnet

Το [owlready2](https://owlready2.readthedocs.io/en/v0.37/) είναι μια βιβλιοθήκη της python για τον χειρισμό και τη δημιουργία owl αρχείων.

In [ ]:
!pip install owlready2

To [NLTK](https://www.nltk.org/) είναι βιβλιοθήκη για την επεξεργασία φυσικής γλώσσας. Εμείς θα το χρησιμοποιήσουμε για να συνδέσουμε τα ονόματα των αντικειμένων με synsets του [wordnet](https://www.nltk.org/howto/wordnet.html)

In [ ]:
from owlready2 import *
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

## Δημιουργία Γνώσης

In [ ]:
onto = get_ontology('http://myontology/')
## ids των 100 πρώτων εικόνων
ids = [k for k in coco.coco][:100]

with onto:

  ## Ορισμός εννοιών για εικόνες και αντικείμενα
  class Image(Thing):
    namespace=onto
    pass

  class DepictedObject(Thing):
    namespace=onto
    pass

  ## Ορισμός ρόλου "hasObject" (ποιες εικόνες περιέχουν ποια αντικείμενα)
  class hasObject(Image>>DepictedObject):
    namespace=onto
    pass

  ## Για κάθε εικόνα
  for im_id in ids:
    ## Δημιουργία individual τύπου "Image"
    im = onto['Image'](str(im_id))

    ## Για κάθε αντικείμενο στην εικόνα:
    for obj_name in coco.coco[im_id]['label_texts']:
      

      ## Εύρεση πρώτου synset στο wordnet
      synsets = wn.synsets(obj_name)

      if len(synsets)<1:
        continue
      
      ## Δημιουργία individual τύπου "Object"
      obj = onto['DepictedObject']()

      ## Σύνδεση της εικόνας με το αντικείμενο μέσω του ρόλου hasObject
      im.hasObject.append(obj)

      synset = wn.synsets(obj_name)[0].name()
      
      ## Αν υπάρχει η αντίστοιχη έννοια στη γνώση
      if onto[synset] in onto.classes():
        ## Ορίζουμε το αντικείμενο obj ως τύπου "synset"
        obj.is_a.append(onto[synset])
        

      ## Αν δεν υπάρχει η αντίστοιχη έννοια στη γνώση, την ορίζουμε, μαζί με τις υπερέννοιές της
      else:
        ## Εύρεση υπερώνυμων
        hyper = lambda s:s.hypernyms()
        hypers = [s.name() for s in list(wn.synset(synset).closure(hyper))]
        hypers = reversed(hypers)

        ## Ορισμός ιεραρχίας εννοιών
        father = Thing
        for h in hypers:
          if onto[h] not in onto.classes():
            with onto:
              cl = types.new_class(h,(father,))
          father = onto[h]
        if onto[synset] not in onto.classes():
          with onto:
            cl = types.new_class(synset,(father,))
        ## Ορίζουμε το αντικείμενο obj ως τύπου "synset"
        with onto:
          obj.is_a.append(onto[synset])
        
  ## Αποθηκεύουμε την οντολογία
  onto.save('myonto.nt',format='ntriples')


# Ερωτήματα στη Γνώση (GraphDB)

Κατεβάζουμε το [GraphDB](https://www.ontotext.com/products/graphdb/graphdb-free/)

Setup -> Repositories -> Create new repository -> GraphDB Free

Ruleset -> OWL 2 RL (optimized) -> Create

Connect to repository

(αριστερά) Import -> Upload RDF files -> επιλέγετε το αρχείο της γνώσης (εδώ myonto.nt) -> import -> import

(αριστερά) SPARQL. Μπορούμε να ζητήσουμε πχ όλες τις εικόνες που περιέχουν καρέκλες

```
prefix my:<http://myontology/>
select ?image where { 
	?image my:hasObject ?o .
    ?o a my:chair.n.01.
}
```

Επειδή έχει τρέξει ο reasoner του GraphDB, "ξέρει" (από το wordnet) πως όλες οι καρέκλες (chair.n.01) είναι καθίσματα (seat.n.03). οπότε μπορούμε να απαντήσουμε και το ερώτημα



```
prefix my:<http://myontology/>
select ?image where { 
	?image my:hasObject ?o .
    ?o a my:seat.n.03.
}

```

Μετά μπορούμε να κατεβάσουμε τα αποτελέσματα (Download as) σε ό,τι μορφή θέλουμε.

Γενικά μπορούμε να κάνουμε πολύ σύνθετα SPARQL ερωτήματα, για τα οποία θα βρείτε πολύ υλικό online.



